In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import requests
from urllib.parse import urlencode

In [2]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_link = 'https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q'
final_url = base_url + urlencode(dict(public_key=public_link))
response = requests.get(final_url)
parse_href = response.json()['href']

In [3]:
data = pd.read_csv(parse_href, parse_dates = ['date'])

In [4]:
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [5]:
data.dtypes # см. типы данных

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [6]:
data.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [7]:
# т.к. в колонке purchase_sum есть пропущенные значения, проверяем, нет ли среди них события 'purchase'
data.query('purchase_sum.isna()').event.unique()

array(['app_start', 'choose_item', 'search', 'tap_basket', 'app_install',
       'register'], dtype=object)

In [8]:
data[data.duplicated()] # см. наличие дубликатов строк

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source


In [9]:
data.date.min()

Timestamp('2020-01-01 00:00:00')

In [10]:
data.date.max()

Timestamp('2020-03-31 00:00:00')

In [11]:
# чтобы было легче группировать пользователей по месяцам, создаю отдельную колонку
data['month_by_active'] = data.date.astype('datetime64[M]')

In [12]:
full_purchaces = data.query('event == "purchase"').groupby(['month_by_active', 'device_id'], as_index=False) \
                .agg({'purchase_sum' : 'count'}).rename(columns={'purchase_sum' : 'count_purshaces'})
full_purchaces.head()

,month_by_active,device_id,count_purshaces
0,2020-01-01,4046,2
1,2020-01-01,4120,1
2,2020-01-01,4125,1
3,2020-01-01,4211,1
4,2020-01-01,4214,1


In [13]:
# дф содержащий информацию о кол-ве платящих пользователей и оплаченных заказов
purchaces_by_month = full_purchaces.groupby('month_by_active', as_index=False) \
                    .agg({'device_id' : 'nunique', 'count_purshaces':'sum'}) \
                    .rename(columns={'device_id' : 'purchasing_users'})

purchaces_by_month

,month_by_active,purchasing_users,count_purshaces
0,2020-01-01,39012,59208
1,2020-02-01,27902,44594
2,2020-03-01,23419,37581


In [14]:
# дф с количеством повторных покупок по месяцам
repeat_paying = full_purchaces.query('count_purshaces >= 2').groupby('month_by_active', as_index=False) \
                        .agg({'device_id' : 'nunique'}).rename(columns={'device_id' : 'users_with_repeat_paying'})
repeat_paying

,month_by_active,users_with_repeat_paying
0,2020-01-01,10456
1,2020-02-01,8622
2,2020-03-01,7073


###### Из датафрейма purchaces_by_month видно, что в январе совершено больше всего покупок и самое большое кол-во пользователей, в том числе тех, кто совершил повторные покупки. 
Но это абсолютные значения и по ним сложно судить, например, о лояльности пользователей к продукту. 

Определим долю пользователей, совершивших повторную покупку по месяцам.

In [15]:
for_conversion = purchaces_by_month.merge(repeat_paying, how='left', on = 'month_by_active')
for_conversion

,month_by_active,purchasing_users,count_purshaces,users_with_repeat_paying
0,2020-01-01,39012,59208,10456
1,2020-02-01,27902,44594,8622
2,2020-03-01,23419,37581,7073


In [16]:
# создаю новую колонку для подсчета доли пользователей, совершивших повторные покупки
for_conversion['cr_to_repeat_purchaces'] = round((for_conversion.users_with_repeat_paying / for_conversion \
                                                  .purchasing_users)*100,2)

In [17]:
for_conversion

,month_by_active,purchasing_users,count_purshaces,users_with_repeat_paying,cr_to_repeat_purchaces
0,2020-01-01,39012,59208,10456,26.8
1,2020-02-01,27902,44594,8622,30.9
2,2020-03-01,23419,37581,7073,30.2


###### Определив CR в повторную покупку, можно утверждать, что максимальное значение в феврале 30,9

###### Рассчитаем MAU по каждому месяцу нашего периода

In [18]:
data.groupby('month_by_active', as_index=False).agg({'device_id' : 'nunique'})

,month_by_active,device_id
0,2020-01-01,99161
1,2020-02-01,75032
2,2020-03-01,74623


###### Определим кол-во установок по месяцам

In [77]:
# В январе кол-во установок в разы больше, чем в феврале или в марте
data.query('event == "app_install"').groupby('month_by_active', as_index=False).agg({'device_id' : 'nunique'})

,month_by_active,device_id
0,2020-01-01,80297
1,2020-02-01,38078
2,2020-03-01,36222


###### Обратим внимание на конверсию из установки в покупку по дням.
Считаем, что пользователь сконвертировался, если с момента установки приложения до первой покупки прошло не более 7 дней

In [19]:
# Создаю датафрейм, где дату установки приложения принимаю за день рожденья пользователя (для формирования когорт)
birthday_hogorts = data[['date', 'event', 'device_id']].query('event == "app_install"').sort_values('date') \
                .drop_duplicates('device_id').rename(columns={'date':'birthday'})
birthday_hogorts.head()

,birthday,event,device_id
2515061,2020-01-01,app_install,4921563
2517441,2020-01-01,app_install,33180892
2517442,2020-01-01,app_install,3331097
2517443,2020-01-01,app_install,1741756
2517444,2020-01-01,app_install,5420901


In [20]:
# Создаю датафрейм, содержащий только первые покупки пользователей
first_purchases = data[['date', 'event', 'device_id']].query('event == "purchase"').sort_values('date') \
                .drop_duplicates('device_id').rename(columns={'date':'first_purchase_day'})

first_purchases.head()

,first_purchase_day,event,device_id
7691,2020-01-01,purchase,6081155
8289,2020-01-01,purchase,1955306
8290,2020-01-01,purchase,20176327
8291,2020-01-01,purchase,28603960
8292,2020-01-01,purchase,14028316


In [21]:
# Создаю дф, который содержит информацию о покупках тех пользователей, у которых есть дата установки приложения
installs_with_purchases = birthday_hogorts.merge(first_purchases[['first_purchase_day', 'device_id']], 
                                                 how='left', on = 'device_id')
installs_with_purchases.head()

,birthday,event,device_id,first_purchase_day
0,2020-01-01,app_install,4921563,2020-01-10
1,2020-01-01,app_install,33180892,NaT
2,2020-01-01,app_install,3331097,2020-01-02
3,2020-01-01,app_install,1741756,2020-01-19
4,2020-01-01,app_install,5420901,2020-01-04


In [22]:
installs_with_purchases.dtypes

birthday              datetime64[ns]
event                         object
device_id                      int64
first_purchase_day    datetime64[ns]
dtype: object

In [23]:
installs_with_purchases['delta_time'] = (installs_with_purchases['first_purchase_day'] - installs_with_purchases['birthday']) \
                                        .dt.days

fast_users = installs_with_purchases.query('delta_time.notna() & delta_time <= 7').groupby('birthday', as_index=False) \
            .agg({'device_id':'nunique'})

# получили число совершивших первую покупку пользователей по дневным когортам
fast_users.head() 

,birthday,device_id
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587


In [25]:
cohorts_info = birthday_hogorts.groupby('birthday', as_index=False).agg({'device_id' : 'nunique'}) \
                .rename(columns={'device_id':'start_users'})

# получили изначальные размеры когорт по дням
cohorts_info.head()

,birthday,start_users
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671


In [26]:
itog = cohorts_info.merge(fast_users, how='left', on = 'birthday')
itog.head()

,birthday,start_users,device_id
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587


In [28]:
# создаю новую колонку для значения CR из установки в покупку
itog['cr_install_to_purchase'] = round((itog.device_id / itog.start_users)*100,2)

In [29]:
itog.sort_values('cr_install_to_purchase', ascending=False).head()

,birthday,start_users,device_id,cr_install_to_purchase
0,2020-01-01,3579,1408,39.34
8,2020-01-09,1424,558,39.19
14,2020-01-15,4310,1650,38.28
13,2020-01-14,5173,1973,38.14
1,2020-01-02,3144,1186,37.72


###### На основании полученных данных, в когорте, сформированной 01.01.2020 из установки в покупку сконвертировалось больше всего  пользователей (в течение первых 7 дней с момента установки), CR = 39.3

###### Также посмотрим, с какого канала приходит больше всего пользователей

In [30]:
data.query('event == "app_install"').groupby('utm_source', as_index=False).agg({'device_id':'nunique'}).sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,32460
6,yandex-direct,29368
2,google_ads,26286
5,vk_ads,23189
3,instagram_ads,20096
1,facebook_ads,13916
4,referal,9282


###### Больше всего новых пользователей пришло от Яндекса

###### Изучим поведение зарегистрированных пользователей и то, как изменялась их активность в каждом из этапов воронки 
* из регистрации в поиск товаров (каталог), 
* из поиска до добавления товара в корзину, 
* из добавления товара в корзину до перехода в корзину, 
* из перехода в корзину до оплаты заказа.

In [31]:
# создаю дф, содержащий дату регистрации для каждого пользователя (который зарегистрировался в период 1 квартала 2020 г.)
register_users = data[['device_id', 'date','event']].query('event == "register"').rename(columns={'date':'register_date'})
register_users.head()

,device_id,register_date,event
2669658,294193,2020-01-01,register
2669659,22917617,2020-01-01,register
2669660,15248490,2020-01-01,register
2669661,252062,2020-01-01,register
2669662,2251583,2020-01-01,register


In [32]:
# создаю дф для воронки
data_for_funnel = data[['date', 'event', 'device_id']] \
                .merge(register_users[['device_id', 'register_date']], how = 'left', on = 'device_id')
data_for_funnel_reg = data_for_funnel.query('register_date.notna()')
data_for_funnel_reg.head()

,date,event,device_id,register_date
0,2020-01-01,app_start,669460,2020-01-07
1,2020-01-01,app_start,833621,2020-01-01
2,2020-01-01,app_start,1579237,2020-01-01
3,2020-01-01,app_start,1737182,2020-01-01
5,2020-01-01,app_start,5148476,2020-01-24


In [33]:
# см. долю перешедших в каталог после регистрации
data_for_funnel_reg.query('register_date <= date & event == "search"').agg({'device_id':'nunique'})/data_for_funnel_reg \
.query('register_date <= date & event=="register"').agg({'device_id':'nunique'})*100

device_id    100.0
dtype: float64

In [34]:
# см. долю пользователей, отправивших товар в корзину после просмотра каталога
data_for_funnel_reg.query('register_date <= date & event == "choose_item"').agg({'device_id':'nunique'})/data_for_funnel_reg \
.query('register_date <= date & event == "search"').agg({'device_id':'nunique'})*100

device_id    100.0
dtype: float64

In [35]:
# см. долю пользователей,перешедших в корзину, после добавления в нее товара
data_for_funnel_reg.query('register_date <= date & event == "tap_basket"').agg({'device_id':'nunique'})/data_for_funnel_reg \
.query('register_date <= date & event == "choose_item"').agg({'device_id':'nunique'})*100

device_id    100.0
dtype: float64

In [36]:
# см. долю оплативших заказ пользователей после перехода в корзину
data_for_funnel_reg.query('register_date <= date & event == "purchase"').agg({'device_id':'nunique'})/data_for_funnel_reg \
.query('register_date <= date & event == "tap_basket"').agg({'device_id':'nunique'})*100

device_id    90.505683
dtype: float64

###### На моменте перехода в корзину отваливаются около 10% зарегистрированных пользователей

###### Обратим внимание на конверсию в первую покупку по источникам
CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые установивших приложение 

In [37]:
paying_users = data.query('event == "purchase"').groupby('utm_source', as_index=False).agg({'device_id':'nunique'}) \
                .rename(columns={'device_id':'paying_users'})
paying_users

,utm_source,paying_users
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [38]:
start_users = data.query('event == "app_start"').groupby('utm_source', as_index=False).agg({'device_id':'nunique'}) \
                .rename(columns={'device_id':'start_users'})
start_users

,utm_source,start_users
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [39]:
sources = paying_users.merge(start_users, how='left', on='utm_source')
sources

,utm_source,paying_users,start_users
0,-,16598,52268
1,facebook_ads,9017,25957
2,google_ads,11339,38092
3,instagram_ads,10762,31045
4,referal,6362,15924
5,vk_ads,12364,34079
6,yandex-direct,12028,40707


In [40]:
sources['cr'] = sources.paying_users/sources.start_users*100

In [41]:
sources.sort_values('cr', ascending=False)

,utm_source,paying_users,start_users,cr
4,referal,6362,15924,39.952273
5,vk_ads,12364,34079,36.280407
1,facebook_ads,9017,25957,34.738221
3,instagram_ads,10762,31045,34.665808
0,-,16598,52268,31.755567
2,google_ads,11339,38092,29.767405
6,yandex-direct,12028,40707,29.547744


###### Пользователи, поступившие по каналу Яндекса меньше всех конвертируются в покупателя.
Лидеры реферальная программа и ВК

###### Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [42]:
# создаю дф, содержащий только первые покупки каждого пользователя
first_purchases = data.query('event=="purchase"').sort_values('date').drop_duplicates('device_id')
first_purchases

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month_by_active
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,2020-01-01
8289,2020-01-01,purchase,292.0,ios,1955306,female,Moscow,yandex-direct,2020-01-01
8290,2020-01-01,purchase,664.0,ios,20176327,female,Moscow,facebook_ads,2020-01-01
8291,2020-01-01,purchase,337.0,android,28603960,male,Moscow,-,2020-01-01
8292,2020-01-01,purchase,502.0,ios,14028316,female,Moscow,referal,2020-01-01
...,...,...,...,...,...,...,...,...,...
2506842,2020-03-31,purchase,670.0,ios,32122725,male,Saint-Petersburg,-,2020-03-01
2506845,2020-03-31,purchase,407.0,ios,23188703,female,Moscow,instagram_ads,2020-03-01
2506848,2020-03-31,purchase,149.5,android,25732558,male,Saint-Petersburg,referal,2020-03-01
2506852,2020-03-31,purchase,442.5,android,17778508,male,Saint-Petersburg,referal,2020-03-01


In [43]:
first_purchases.groupby('utm_source', as_index=False).agg({'purchase_sum':'median'}).round(2).sort_values('purchase_sum')

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


###### У пользователей, пришедших по реферальной программе, медианный чек (по первой покупке) выше

###### Вычислим значение ROMI для каждого платного канала 

###### Данные по затратам на рекламу:

- Яндекс – 10 491 707 руб.
- Гугл – 10 534 878 руб.
- Фейсбук – 8 590 498 руб.
- Инстаграм – 8 561 626 руб.
- ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

In [44]:
ya = 10491707
google = 10534878
fb = 8590498
inst = 8561626
vk = 9553531

In [45]:
revenue = data.groupby('utm_source', as_index=False).agg({'purchase_sum':'sum'})
revenue

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [46]:
ROMI_ya = (revenue.query('utm_source == "yandex-direct"').purchase_sum - ya) / ya
ROMI_ya

6    0.326321
Name: purchase_sum, dtype: float64

In [47]:
ROMI_google = (revenue.query('utm_source == "google_ads"').purchase_sum - google) / google
ROMI_google

2    0.221493
Name: purchase_sum, dtype: float64

In [48]:
ROMI_fb = (revenue.query('utm_source == "facebook_ads"').purchase_sum - fb) / fb
ROMI_fb

1    0.425983
Name: purchase_sum, dtype: float64

In [49]:
ROMI_inst = (revenue.query('utm_source == "instagram_ads"').purchase_sum - inst) / inst
ROMI_inst

3    0.69909
Name: purchase_sum, dtype: float64

In [50]:
ROMI_vk = (revenue.query('utm_source == "vk_ads"').purchase_sum - vk) / vk
ROMI_vk

5    0.71556
Name: purchase_sum, dtype: float64

In [51]:
first_purchases_by_referal =  data.query('utm_source == "referal" & event == "purchase"').sort_values('date') \
                            .drop_duplicates('device_id')

first_purchases_by_referal

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month_by_active
7722,2020-01-01,purchase,289.5,android,26467125,male,Saint-Petersburg,referal,2020-01-01
8134,2020-01-01,purchase,583.5,ios,6100382,female,Moscow,referal,2020-01-01
8173,2020-01-01,purchase,178.5,android,23519113,male,Saint-Petersburg,referal,2020-01-01
8198,2020-01-01,purchase,189.5,ios,14747684,male,Moscow,referal,2020-01-01
8212,2020-01-01,purchase,561.5,ios,13926829,male,Moscow,referal,2020-01-01
...,...,...,...,...,...,...,...,...,...
2506874,2020-03-31,purchase,357.0,android,11203331,female,Moscow,referal,2020-03-01
2506852,2020-03-31,purchase,442.5,android,17778508,male,Saint-Petersburg,referal,2020-03-01
2506848,2020-03-31,purchase,149.5,android,25732558,male,Saint-Petersburg,referal,2020-03-01
2506822,2020-03-31,purchase,516.5,android,1440834,male,Moscow,referal,2020-03-01


In [52]:
ROMI_ref = (first_purchases_by_referal.purchase_sum.sum() - first_purchases_by_referal.device_id \
            .nunique()*200) / (first_purchases_by_referal.device_id.nunique()*200)

ROMI_ref

2.4701481452373466

###### Исходя из того, что реферальная программа не является маркетинговым каналом,  то самый высокий ROMI у ВК